In [1]:
import sys
import os
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
from connector import spark_connector

In [3]:
import pandas as pd
import numpy as np
from pyspark.sql import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [4]:
spark_connector?

Init signature:
spark_connector(
    exec_inst: int = 100,
    exec_cores: int = 5,
    aloc_max: int = 500,
    memory_exec: int = 25,
    name: object = 'utopialvo',
    rootdir: object = 'remote_dir_in_hdft',
    enable_graphframes: bool = False,
    enable_clickhouse: bool = False,
    enable_postgres: bool = False,
    dynamic_allocation: bool = False,
    intensive_mode: bool = False,
    marshal_mode: bool = True,
    yarn: bool = True,
) -> None
Docstring:     
Класс стандартного подключения к Spark

exec_inst - количество контейнеров
exec_cores - количество ядер в контейнере
aloc_max - максимальное количество ядер для dynamic_allocation (если он вкл)
memory_exec - количество памяти в контейнере
name - название подключения для упрощения логгирования подключений в кластере
rootdir - корневая папка в HDFS 
enable_graphframes - включить работу с graphframes
enable_clickhouse - включить работу с clickhouse
enable_postgres - включить работу с postgres
dynamic_allocation - включить д

In [5]:
con = spark_connector(exec_inst=100, exec_cores=5, memory_exec=25, enable_graphframes=True, marshal_mode=False, yarn = False)

In [6]:
con.spark

In [7]:
from sklearn.datasets import make_blobs

In [8]:
X, y, c = make_blobs(n_samples=1000, n_features=5, centers=2, cluster_std=1.0, center_box=(-15.0, 15.0), shuffle=True, random_state=0, return_centers=True)

In [9]:
data = pd.DataFrame(X, columns=[f'x{i+1}' for i in range(X.shape[1])])
data['y'] = y
data = data.merge(pd.DataFrame(c, columns=[f'centroids_{i+1}' for i in range(X.shape[1])]).reset_index(), left_on='y', right_on='index', how='left').drop('index', axis=1)

In [10]:
data

,x1,x2,x3,x4,x5,y,centroids_1,centroids_2,centroids_3,centroids_4,centroids_5
0,0.756020,6.439902,2.591171,2.300685,-1.746211,0,1.464405,6.455681,3.082901,1.346495,-2.290356
1,0.481287,5.557359,3.884953,-0.500036,-1.686219,0,1.464405,6.455681,3.082901,1.346495,-2.290356
2,4.603995,-2.926113,13.284943,13.583389,-2.029909,1,4.376823,-1.872384,11.753190,13.909883,-3.496754
3,2.067113,5.861031,2.826944,0.998449,-3.072723,0,1.464405,6.455681,3.082901,1.346495,-2.290356
4,-0.118281,7.416238,3.308742,0.796997,-3.388927,0,1.464405,6.455681,3.082901,1.346495,-2.290356
...,...,...,...,...,...,...,...,...,...,...,...
995,4.315838,-1.034867,11.589772,14.733438,-2.576122,1,4.376823,-1.872384,11.753190,13.909883,-3.496754
996,1.925220,7.379478,2.950333,1.057490,-4.288995,0,1.464405,6.455681,3.082901,1.346495,-2.290356
997,1.527667,6.612188,3.315082,0.749179,-2.528278,0,1.464405,6.455681,3.082901,1.346495,-2.290356
998,1.061228,7.678126,3.291176,2.323135,-1.933990,0,1.464405,6.455681,3.082901,1.346495,-2.290356


In [11]:
feature = con.spark.createDataFrame(data[['x1','x2','x3','x4','x5']].reset_index(names='id'))

In [12]:
feature

DataFrame[id: bigint, x1: double, x2: double, x3: double, x4: double, x5: double]

In [13]:
from stack import SparkSimilarMatrix, GeneratorPysparkDf, BatchedStreamProcessor

In [14]:
SparkSimilarMatrix?

Init signature:
SparkSimilarMatrix(
    spark: pyspark.sql.session.SparkSession,
    train_df: pyspark.sql.dataframe.DataFrame,
) -> None
Docstring:     
Вычисление матрицы схожести в распределенном режиме.

Пример использования:
>>> sim = SparkSimilarMatrix(spark, train_df)
>>> sim.prep()
>>> sim.create_sim_matrix(metric='cosine')
>>> sim.create_degree_matrix(treshold=0.75)
File:           ~/work/Spark_conn/src/stack.py
Type:           type
Subclasses:     

In [15]:
sim = SparkSimilarMatrix(con.spark, feature)
sim.prep()
sim.create_sim_matrix(metric='cosine')
sim.create_degree_matrix(treshold=0.75)

In [16]:
sim.sim_matrix_filtred = sim.sim_matrix_filtred.select(F.col('id_A').alias('id'), 'id_B','distance')

In [17]:
sim.sim_matrix = sim.sim_matrix.select(F.col('id_A').alias('id'), 'id_B','distance')

In [18]:
feature = feature.checkpoint()
sim.sim_matrix = sim.sim_matrix.checkpoint()

In [19]:
feature.createOrReplaceGlobalTempView('feature')
sim.sim_matrix.createOrReplaceGlobalTempView('sim_matrix')

In [20]:
def process_batch(batch):
    print(batch[0], batch[1].shape, batch[2].shape)

In [21]:
bpdf = GeneratorPysparkDf(con.spark, df ='global_temp.feature', df_adj ='global_temp.sim_matrix')
gen = bpdf.get_df_generator(chunk_size=256,idcol='id', num_epoch=6)
processsor = BatchedStreamProcessor(con.spark, buffer_size=10)

In [22]:
for batch in processsor.batched_stream(gen):
    process_batch(batch)
    print('end')
    del batch

2 (256, 6) (129282, 3)
end
1 (256, 6) (128287, 3)
end
2 (256, 6) (127649, 3)
end
1 (232, 6) (110280, 3)
end
2 (256, 6) (125297, 3)
end
1 (256, 6) (135755, 3)
end
1 (256, 6) (125178, 3)
end
2 (232, 6) (117272, 3)
end
4 (256, 6) (135551, 3)
end
4 (256, 6) (123592, 3)
end
3 (256, 6) (126729, 3)
end
3 (256, 6) (129908, 3)
end
3 (256, 6) (127805, 3)
end
4 (232, 6) (115728, 3)
end
3 (232, 6) (115058, 3)
end
4 (256, 6) (124629, 3)
end
5 (256, 6) (125432, 3)
end
5 (256, 6) (128751, 3)
end
6 (256, 6) (131722, 3)
end
5 (232, 6) (119334, 3)
end
5 (256, 6) (125983, 3)
end
6 (256, 6) (123307, 3)
end
6 (256, 6) (126321, 3)
end
6 (232, 6) (118150, 3)
end


In [24]:
#con.stop_spark()

time: 2025-02-22 21:51:48.286801
dont del
stop
